In [1]:
import copy
import os
import pickle

import pandas as pd

In [2]:
import collections

In [3]:
texts = {}
trees = {}
results = {}
names = os.listdir('../русский авторазметчик эде')
for name in names:
    with open('../русский авторазметчик эде' + '/' + name, 'br') as f:
            obj = pickle.load(f)
            if 'tree' in name:
                trees[name] = obj
            else:
                if 'result' in name:
                    results[name] = obj
                else:
                    texts[name] = obj

In [4]:
results[list(results.keys())[0]]['sentences'][1].begin

12

In [5]:
str(results[list(results.keys())[0]]['tokens'][0])

'<0, 2>'

In [6]:
def make_dict(tokens, objects):
    straight_objects = []
    for sentence in objects:
        for object_ in sentence:
            straight_objects.append(object_)
    strings = []
    for token in tokens:
        strings.append(str(token))
    return dict(zip(strings, straight_objects))

In [7]:
def match(result):
    text = results[key]['text']
    tokens = results[key]['tokens']
    sentences = results[key]['sentences']
    lemmas = results[key]['lemma']
    pos = results[key]['postag']
    udpos = results[key]['ud_postag']
    #morph = results[key]['morph']
    sentences_dict = {}
    for i in range(len(tokens)):
        token = tokens[i]
        for j in range(len(sentences)):
            sentence = sentences[j]
            if sentence.begin <= i and i <= sentence.end:
                sentences_dict[str(token)] = j
    lemmas_dict = make_dict(tokens, lemmas)
    pos_dict = make_dict(tokens, pos)
    udpos_dict = make_dict(tokens, udpos)
    #morph_dict = make_dict(tokens, morph)
    all_dicts = {'sentences' : sentences_dict, 'lemmas' : lemmas_dict, 'pos' : pos_dict, 'udpos' : udpos_dict}
    return all_dicts

In [8]:
def match_leaf(all_, tree, tokens):
    words = []
    for token in tokens:
        if tree.start <= token.begin and token.end <= tree.end:
            words.append(token)
    match = dict()
    for word in words:
        for key in all_:
            if key in match:
                match[key].append(all_[key][str(word)])
            else:
                match[key] = []
                match[key].append(all_[key][str(word)])
    match['tokens'] = words
    return match

In [9]:
def segmentation(result, tree, text, id_, tokens_c, all_):
    if tree.left:
        segmentation(result, tree.left, text, id_, tokens_c, all_)
        segmentation(result, tree.right, text, id_, tokens_c, all_)
    else:
        match = match_leaf(all_, tree, tokens_c)
        ids.append(id_)
        sentences.append(match['sentences'])
        tokens.append(match['tokens'])
        lemmas.append(match['lemmas'])
        pos.append(match['pos'])
        udpos.append(match['udpos'])
        #morph.append(match['morph'])
        segments.append(text[tree.start:tree.end])

In [10]:
segments =  []
sentences = []
ids = []
tokens = []
lemmas = []
pos = []
udpos = []
morph = []
for key in results:
#for i in range(2):
    #key = list(results.keys())[i]
    result = results[key]
    id_ = key.split('_')[-1].split('.')[0]
    tree = results[key]['rst'][0]
    text = results[key]['text']
    tokens_c = results[key]['tokens']
    all_ = match(result)
    segmentation(result, tree, text, id_, tokens_c, all_)#, morph)

In [11]:
df = pd.DataFrame({'text_id' :ids, 'sentence' : sentences, 'segment' : segments, 'tokens' : tokens, 'lemmas' : lemmas, 'pos' : pos, 'udpos' : udpos})

In [12]:
df['sentence'] = df['sentence'].apply(lambda x: x[0])

In [13]:
df

,text_id,sentence,segment,tokens,lemmas,pos,udpos
0,b001,0,"Да, очень раздражает","[<0, 2>, <2, 3>, <4, 9>, <10, 20>]","[да, ,, очень, раздражает]","[CONJ, , ADV, VERB]","[PART, PUNCT, ADV, VERB]"
1,b001,0,и утомляет все время сортировать мусор правильно.,"[<21, 22>, <23, 31>, <32, 35>, <36, 41>, <42, ...","[и, утомляет, весь, время, сортировать, мусор,...","[CONJ, VERB, PRON, NOUN, VERB, NOUN, ADV, ]","[CCONJ, VERB, DET, NOUN, VERB, NOUN, ADV, PUNCT]"
2,b001,1,На кухне воняют три разных мешка для мусора.,"[<71, 73>, <74, 79>, <80, 86>, <87, 90>, <91, ...","[на, кухне, воняют, три, разный, мешка, для, м...","[ADP, NOUN, VERB, NUM, ADJ, NOUN, ADP, NOUN, ]","[ADP, NOUN, VERB, NUM, ADJ, NOUN, ADP, NOUN, P..."
3,b001,2,Их нужно отсортировать в разные мусорные баки.,"[<116, 118>, <119, 124>, <125, 138>, <139, 140...","[их, нужно, отсортировать, в, разный, мусорные...","[PRON, ADV, VERB, ADP, ADJ, ADJ, NOUN, ]","[PRON, VERB, VERB, ADP, ADJ, ADJ, NOUN, PUNCT]"
4,b001,3,Но все же Германия производит слишком много му...,"[<163, 165>, <166, 169>, <170, 172>, <173, 181...","[но, весь, же, германия, производить, слишком,...","[CONJ, PRON, PART, NOUN, VERB, ADV, ADV, NOUN]","[CCONJ, PART, PART, PROPN, VERB, ADV, NUM, NOUN]"
...,...,...,...,...,...,...,...
749,k031,2,"по мнению общества,","[<207, 209>, <210, 216>, <217, 225>, <225, 226>]","[по, мнение, общество, ,]","[ADP, NOUN, NOUN, ]","[ADP, NOUN, NOUN, PUNCT]"
750,k031,2,"кажется довольно деспотичным методом наказания,","[<227, 234>, <235, 243>, <244, 255>, <256, 263...","[кажется, довольно, деспотичным, метод, наказа...","[VERB, ADV, ADJ, NOUN, NOUN, ]","[VERB, ADV, ADJ, NOUN, NOUN, PUNCT]"
751,k031,2,"но, в конце концов, подобные правила часто рас...","[<275, 277>, <277, 278>, <279, 280>, <281, 286...","[но, ,, в, конец, конец, ,, подобный, правило,...","[CONJ, , ADP, NOUN, NOUN, , ADJ, NOUN, ADV, VE...","[CCONJ, PUNCT, ADP, NOUN, NOUN, PUNCT, ADJ, NO..."
752,k031,3,Поэтому увеличение штрафов – это верный путь.,"[<391, 398>, <399, 409>, <410, 417>, <418, 419...","[поэтому, увеличение, штрафов, –, это, верный,...","[ADV, NOUN, NOUN, , PART, ADJ, NOUN, ]","[SCONJ, NOUN, NOUN, PUNCT, AUX, ADJ, NOUN, PUNCT]"


In [14]:
import pickle

In [15]:
with open('segments_with_tags.pickle', 'wb') as f:
    pickle.dump(df, f)

In [16]:
df_2 = df.copy()

In [17]:
def tok(a):
    a_2 = []
    for t in a:
        a_2.append(str(t))
    return '_'.join(a_2)

In [18]:
df_2['tokens'] = df_2['tokens'].apply(tok)
for name in ['lemmas', 'pos', 'udpos']:
    df_2[name] = df_2[name].apply(lambda x: '_'.join(x))

In [19]:
df_2.to_excel('segments_with_tags.xlsx')

In [20]:
df_2

,text_id,sentence,segment,tokens,lemmas,pos,udpos
0,b001,0,"Да, очень раздражает","<0, 2>_<2, 3>_<4, 9>_<10, 20>","да_,_очень_раздражает",CONJ__ADV_VERB,PART_PUNCT_ADV_VERB
1,b001,0,и утомляет все время сортировать мусор правильно.,"<21, 22>_<23, 31>_<32, 35>_<36, 41>_<42, 53>_<...",и_утомляет_весь_время_сортировать_мусор_правил...,CONJ_VERB_PRON_NOUN_VERB_NOUN_ADV_,CCONJ_VERB_DET_NOUN_VERB_NOUN_ADV_PUNCT
2,b001,1,На кухне воняют три разных мешка для мусора.,"<71, 73>_<74, 79>_<80, 86>_<87, 90>_<91, 97>_<...",на_кухне_воняют_три_разный_мешка_для_мусора_.,ADP_NOUN_VERB_NUM_ADJ_NOUN_ADP_NOUN_,ADP_NOUN_VERB_NUM_ADJ_NOUN_ADP_NOUN_PUNCT
3,b001,2,Их нужно отсортировать в разные мусорные баки.,"<116, 118>_<119, 124>_<125, 138>_<139, 140>_<1...",их_нужно_отсортировать_в_разный_мусорные_бак_.,PRON_ADV_VERB_ADP_ADJ_ADJ_NOUN_,PRON_VERB_VERB_ADP_ADJ_ADJ_NOUN_PUNCT
4,b001,3,Но все же Германия производит слишком много му...,"<163, 165>_<166, 169>_<170, 172>_<173, 181>_<1...",но_весь_же_германия_производить_слишком_много_...,CONJ_PRON_PART_NOUN_VERB_ADV_ADV_NOUN,CCONJ_PART_PART_PROPN_VERB_ADV_NUM_NOUN
...,...,...,...,...,...,...,...
749,k031,2,"по мнению общества,","<207, 209>_<210, 216>_<217, 225>_<225, 226>","по_мнение_общество_,",ADP_NOUN_NOUN_,ADP_NOUN_NOUN_PUNCT
750,k031,2,"кажется довольно деспотичным методом наказания,","<227, 234>_<235, 243>_<244, 255>_<256, 263>_<2...","кажется_довольно_деспотичным_метод_наказание_,",VERB_ADV_ADJ_NOUN_NOUN_,VERB_ADV_ADJ_NOUN_NOUN_PUNCT
751,k031,2,"но, в конце концов, подобные правила часто рас...","<275, 277>_<277, 278>_<279, 280>_<281, 286>_<2...","но_,_в_конец_конец_,_подобный_правило_часто_ра...",CONJ__ADP_NOUN_NOUN__ADJ_NOUN_ADV_VERB_PRON_NO...,CCONJ_PUNCT_ADP_NOUN_NOUN_PUNCT_ADJ_NOUN_ADV_V...
752,k031,3,Поэтому увеличение штрафов – это верный путь.,"<391, 398>_<399, 409>_<410, 417>_<418, 419>_<4...",поэтому_увеличение_штрафов_–_это_верный_путь_.,ADV_NOUN_NOUN__PART_ADJ_NOUN_,SCONJ_NOUN_NOUN_PUNCT_AUX_ADJ_NOUN_PUNCT
